In [1]:
import os
import time
from PIL import Image
from torchvision import transforms, models
from torch.utils.data import DataLoader, Dataset, random_split
import pytorch_lightning as pl
import torch
from sklearn.metrics import accuracy_score
from tqdm import tqdm

In [2]:
# Dataset Class
class PathfinderDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        try:
            image = Image.open(img_path).convert('RGB')
            if self.transform:
                image = self.transform(image)
            return image, label
        except (IOError, OSError, Image.DecompressionBombError, Image.UnidentifiedImageError):
            new_idx = (idx + 1) % len(self.data)
            return self[new_idx]

In [3]:
# Data Module
class PathfinderDataModule(pl.LightningDataModule):
    def __init__(self, dataset_dir, batch_size=32):
        super().__init__()
        self.dataset_dir = dataset_dir
        self.batch_size = batch_size
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
        ])

    def prepare_data(self):
        print("Preparing data...")
        self.data_list = []
        metadata_dir = os.path.join(self.dataset_dir, 'metadata')
        for file_name in os.listdir(metadata_dir):
            metadata_path = os.path.join(metadata_dir, file_name)
            with open(metadata_path, 'r') as file:
                for line in file:
                    tokens = line.strip().split()
                    img_rel_path = tokens[0] + "/" + tokens[1]
                    label = int(tokens[3])
                    img_path = os.path.join(self.dataset_dir, img_rel_path)
                    self.data_list.append((img_path, label))
        print(f"Prepared {len(self.data_list)} data entries.")

    def setup(self, stage=None):
        print("Setting up datasets...")
        dataset = PathfinderDataset(self.data_list, transform=self.transform)

        train_size = int(0.8 * len(dataset))
        val_size = int(0.1 * len(dataset))
        test_size = len(dataset) - train_size - val_size
        self.train_set, self.val_set, self.test_set = random_split(
            dataset, [train_size, val_size, test_size])

    def train_dataloader(self):
        return DataLoader(self.train_set, batch_size=self.batch_size, shuffle=True, num_workers=4)

    def val_dataloader(self):
        return DataLoader(self.val_set, batch_size=self.batch_size, num_workers=4)

    def test_dataloader(self):
        return DataLoader(self.test_set, batch_size=self.batch_size, num_workers=4)

In [4]:
# Define Dataset Directory and Data Module
dataset_dir = '/kaggle/input/lra-pathfinder-32/pathfinder32/curv_contour_length_14'  # Update as needed
data_module = PathfinderDataModule(dataset_dir, batch_size=32)

# Prepare Data
data_module.prepare_data()
data_module.setup()

# Define Model
print("Initializing model...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.mobilenet_v2(pretrained=True)  # Use MobileNetV2
model.classifier[1] = torch.nn.Linear(model.last_channel, 2)  # Update for binary classification
model.to(device)

Preparing data...
Prepared 200000 data entries.
Setting up datasets...
Initializing model...


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 121MB/s]


MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [5]:
# Training Function
def train_model(model, train_loader, optimizer, loss_fn, epochs=5):
    print("Starting training...")
    model.train()
    start_time = time.time()
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        total_loss = 0
        for images, labels in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
        print(f"Epoch {epoch+1} Loss: {total_loss / len(train_loader):.4f}")
    end_time = time.time()
    training_time = end_time - start_time
    return training_time

In [6]:
# Evaluation Function
def evaluate_model(model, test_loader):
    print("Evaluating model...")
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Evaluating"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=-1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    accuracy = accuracy_score(all_labels, all_preds)
    print(f"Test Accuracy: {accuracy:.4f}")
    return accuracy

In [7]:
# Train and Evaluate
print("Training and evaluation begin...")
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss()

train_loader = data_module.train_dataloader()
test_loader = data_module.test_dataloader()

training_time = train_model(model, train_loader, optimizer, loss_fn, epochs=5)
accuracy = evaluate_model(model, test_loader)

Training and evaluation begin...
Starting training...
Epoch 1/5


Training Epoch 1: 100%|██████████| 5000/5000 [10:35<00:00,  7.86it/s]


Epoch 1 Loss: 0.6829
Epoch 2/5


Training Epoch 2: 100%|██████████| 5000/5000 [10:38<00:00,  7.83it/s]


Epoch 2 Loss: 0.5458
Epoch 3/5


...

Epoch 4 Loss: 0.4143
Epoch 5/5


Training Epoch 5: 100%|██████████| 5000/5000 [10:40<00:00,  7.81it/s]


Epoch 5 Loss: 0.3711
Evaluating model...


Evaluating: 100%|██████████| 625/625 [00:57<00:00, 10.93it/s]

Test Accuracy: 0.7968


In [8]:
# Calculate Metrics
num_params = sum(p.numel() for p in model.parameters())
efficiency = accuracy / (torch.log(torch.tensor(training_time)) * torch.log(torch.tensor(num_params)))

# Print Metrics
print("\nTraining Metrics:")
print(f"Time Taken: {training_time:.2f} seconds")
print(f"Accuracy: {accuracy:.4f}")
print(f"Number of Parameters: {num_params}")
print(f"Efficiency Score: {efficiency:.4f}")


Training Metrics:
Time Taken: 3193.16 seconds
Accuracy: 0.7968
Number of Parameters: 2226434
Efficiency Score: 0.0068
